In [22]:
#Imports
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge
import xgboost as xgb

   ---------------------------------------- 99.8/99.8 MB 1.5 MB/s eta 0:00:00


In [30]:
import csv
df=csv.reader("data/train.csv")
df.drop(columns=["id", "amenities", "description", "cleaning_fee", "city", "first_review", "host_has_profile_pic", "host_identity_verified", "host_response_rate", "host_since", "instant_bookable", "last_review", "latitude", "longitude", "name", "number_of_reviews", "review_scores_rating", "thumbnail_url", "zipcode", "neighbourhood"], inplace=True)
#df.to_csv("data/cleaned_train.csv", index=False)
df.head()

AttributeError: '_csv.reader' object has no attribute 'drop'

In [4]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("data/cleaned_train.csv")
df.head(5)

,log_price,property_type,room_type,accommodates,bathrooms,bed_type,cancellation_policy,bedrooms,beds,wireless_internet,kitchen,heating,essentials,smoke_detector,air_conditioning,tv
0,5.010635,Apartment,Entire home/apt,3,1.0,Real Bed,strict,1.0,1.0,True,True,True,True,False,True,False
1,5.129899,Apartment,Entire home/apt,7,1.0,Real Bed,strict,3.0,3.0,True,True,True,True,True,True,False
2,4.976734,Apartment,Entire home/apt,5,1.0,Real Bed,moderate,1.0,3.0,True,True,True,True,True,True,True
3,6.620073,House,Entire home/apt,4,1.0,Real Bed,flexible,2.0,2.0,True,True,True,True,True,False,True
4,4.744932,Apartment,Entire home/apt,2,1.0,Real Bed,moderate,0.0,1.0,True,True,True,True,True,True,True


In [5]:
#fill null values
df['beds'] = df['beds'].fillna(1)
df['bedrooms'] = df['bedrooms'].fillna(1)
df['bathrooms'] = df['bathrooms'].fillna(1)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74111 entries, 0 to 74110
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   log_price            74111 non-null  float64
 1   property_type        74111 non-null  object 
 2   room_type            74111 non-null  object 
 3   accommodates         74111 non-null  int64  
 4   bathrooms            74111 non-null  float64
 5   bed_type             74111 non-null  object 
 6   cancellation_policy  74111 non-null  object 
 7   bedrooms             74111 non-null  float64
 8   beds                 74111 non-null  float64
 9   wireless_internet    74111 non-null  bool   
 10  kitchen              74111 non-null  bool   
 11  heating              74111 non-null  bool   
 12  essentials           74111 non-null  bool   
 13  smoke_detector       74111 non-null  bool   
 14  air_conditioning     74111 non-null  bool   
 15  tv                   74111 non-null 

In [12]:
# Assuming df is your DataFrame containing the categorical variables and 'price'
# Let's encode the categorical variables using LabelEncoder
le = LabelEncoder()
df_encoded = df.copy()  # Create a copy of the DataFrame to avoid modifying the original data

for col in df.select_dtypes(include=['object', 'bool']):
    df_encoded[col] = le.fit_transform(df[col])

df_encoded.head()

,log_price,property_type,room_type,accommodates,bathrooms,bed_type,cancellation_policy,bedrooms,beds,wireless_internet,kitchen,heating,essentials,smoke_detector,air_conditioning,tv
0,5.010635,0,0,3,1.0,4,2,1.0,1.0,1,1,1,1,0,1,0
1,5.129899,0,0,7,1.0,4,2,3.0,3.0,1,1,1,1,1,1,0
2,4.976734,0,0,5,1.0,4,1,1.0,3.0,1,1,1,1,1,1,1
3,6.620073,17,0,4,1.0,4,0,2.0,2.0,1,1,1,1,1,0,1
4,4.744932,0,0,2,1.0,4,1,0.0,1.0,1,1,1,1,1,1,1


In [15]:
df_train, df_test_cv  = train_test_split(df,test_size=0.25,random_state = 101)

In [20]:
lin_reg = LinearRegression()
X=df_encoded.drop(['log_price'],axis=1)
Y=df_encoded['log_price']
kf = KFold(n_splits=5, shuffle=True, random_state = 101)
print(cross_val_score(lin_reg, X, Y, # estimator, features, target
                cv=5, # number of folds 
                scoring='r2'))
      
print(np.mean(cross_val_score(lin_reg, X, Y, cv=kf, scoring='r2')))

[0.51850726 0.52605851 0.52248722 0.50996426 0.51228973]
0.5178240655342705


In [24]:
#XGBRegressor
lm_xgb = xgb.XGBRegressor()

print(cross_val_score(lm_xgb, X, Y, # estimator, features, target
                cv=5, # number of folds 
                scoring='r2'))

print(np.mean(cross_val_score(lm_xgb, X, Y, cv=kf, scoring='r2')))

[0.55243973 0.55714063 0.56087137 0.54601563 0.5464098 ]
0.5513339122145291


In [26]:
lm_reg = Ridge(alpha=1)

print(cross_val_score(lm_reg, X, Y, # estimator, features, target
                cv=5, # number of folds 
                scoring='r2'))

print(np.mean(cross_val_score(lm_reg, X, Y, cv=kf, scoring='r2')))

[0.51850717 0.526058   0.52248635 0.50996566 0.51229001]
0.5178241022355883
